In [1]:
%defaultDatasource jdbc:h2:mem:db

# Modelo Conceitual.
**Adicionar Imagem**





# Modelo Lógico.
**Adicionar Imagem**



Paciente(_id_, idade, sexo, eh_fumante [B], hist_diabetes[B], hist_doenca_arterial[B])
Primary Key: id

Sintoma(_id_paciente_, local_dor[str], provocada_por_esforço[B], melhorou_com_descanso[B], tipo_dor_toracica[str])
Primary Key: id_paciente
Foreigner Key: id_paciente

ECG(_id_paciente_, resultado, protocolo_exercicio, duracao_exercicio, batimento_max, pico_pressao)
Primary Key: id_paciente
Foreigner Key: id_paciente

Triagem(_id_paciente_, pressao_repouso, colesterol, glicose)
Primary Key: id_paciente
Foreigner Key: id_paciente

Diagnostico(_id_paciente_, status_angiografico, vaso_atingido)
Primary Key: id_paciente
Foreigner Key: id_paciente

In [2]:
--Task List:


--Paciente(_id_, idade, sexo, eh_fumante [B], hist_diabetes[B], hist_doenca_arterial[B])
--Primary Key: id


DROP TABLE IF EXISTS Paciente;
DROP TABLE IF EXISTS Sintoma;
DROP TABLE IF EXISTS ECG;
DROP TABLE IF EXISTS Triagem;
DROP TABLE IF EXISTS Diagnostico;

CREATE TABLE Paciente (
id VARCHAR(36),
idade FLOAT,
genero FLOAT, 
PRIMARY KEY(id)
) AS SELECT
    uid,
    idade,
    genero
FROM CSVREAD('cleveland_id.csv');


--Sintoma(_id_paciente_, local_dor[str], provocada_por_esforço[B], melhorou_com_descanso[B], tipo_dor_toracica[str])
--Primary Key: id_paciente
--Foreigner Key: id_paciente

CREATE TABLE Sintoma (
id_paciente VARCHAR(36),
tipo_dor_toracica FLOAT,
-- local_dor VARCHAR(100),
-- provocada_por_esforço INTEGER,   
-- melhorou_com_descanso VARCHAR(100),
-- tipo_dor_toracica VARCHAR(100),  
PRIMARY KEY(id_paciente),
FOREIGN KEY(id_paciente)
    REFERENCES Paciente(id)
      ON DELETE NO ACTION
      ON UPDATE NO ACTION    
) AS SELECT
    uid,
    tipo_dor_toracica
FROM CSVREAD('cleveland_id.csv');


--ECG(_id_paciente_, resultado, protocolo_exercicio, duracao_exercicio, batimento_max, pico_pressao)
--Primary Key: id_paciente
--Foreigner Key: id_paciente
    
CREATE TABLE ECG (
id_paciente VARCHAR(36),
resultado FLOAT,
batimento_max FLOAT,
induziu_angina FLOAT,
depressao_st FLOAT,
declive_st FLOAT,
numero_vasos FLOAT,
thal FLOAT,
PRIMARY KEY(id_paciente),
FOREIGN KEY(id_paciente)
    REFERENCES Paciente(id)
      ON DELETE NO ACTION
      ON UPDATE NO ACTION  

) AS SELECT
    uid,
    resultado,
    batimento_max,
    induziu_angina,
    depressao_st,
    declive_st,
    numero_vasos,
    thal
FROM CSVREAD('cleveland_id.csv');


--Triagem(_id_paciente_, pressao_repouso, colesterol, glicose)
--Primary Key: id_paciente
--Foreigner Key: id_paciente


CREATE TABLE Triagem (
id_paciente VARCHAR(36),
pressao_repouso FLOAT,
colesterol FLOAT,
glicose FLOAT,
PRIMARY KEY(id_paciente),
FOREIGN KEY(id_paciente)
REFERENCES Paciente(id)
      ON DELETE NO ACTION
      ON UPDATE NO ACTION  

) AS SELECT
    uid,
    pressao_repouso,
    colesterol,
    glicose
FROM CSVREAD('cleveland_id.csv');

--Diagnostico(_id_paciente_, status_angiografico, vaso_atingido)
--Primary Key: id_paciente
--Foreigner Key: id_paciente


CREATE TABLE Diagnostico (
id_paciente VARCHAR(36),
diagnostico FLOAT,
PRIMARY KEY(id_paciente),
FOREIGN KEY(id_paciente)
REFERENCES Paciente(id)
      ON DELETE NO ACTION
      ON UPDATE NO ACTION  

) AS SELECT
    uid,
    diagnostico
FROM CSVREAD('cleveland_id.csv');


In [4]:
SELECT * FROM Paciente;

In [5]:
SELECT * FROM Sintoma;

In [6]:
SELECT * FROM ECG;

In [42]:
SELECT COUNT(*) FROM Paciente P, Diagnostico D WHERE D.diagnostico = 0 AND D.id_paciente = P.id;

164

In [3]:
SELECT COUNT(*) FROM Paciente P, Diagnostico D WHERE D.diagnostico > 0 AND D.id_paciente = P.id;

139

In [27]:
SELECT AVG(Triagem.pressao_repouso) FROM Triagem

131.68976897689768

In [25]:
SELECT COUNT(*)
FROM Diagnostico D, Paciente P, Triagem T
WHERE (D.diagnostico > 0 AND D.id_paciente = P.id) AND (T.id_paciente = P.id AND T.pressao_repouso > (SELECT AVG(Triagem.pressao_repouso) FROM Triagem));

68

In [26]:
SELECT COUNT(*)
FROM Diagnostico D, Paciente P, Triagem T
WHERE (D.diagnostico > 0 AND D.id_paciente = P.id) AND (T.id_paciente = P.id AND T.pressao_repouso > 120.0);

102

In [28]:
SELECT AVG(Triagem.colesterol) FROM Triagem

246.69306930693068

In [29]:
SELECT COUNT(*)
FROM Diagnostico D, Paciente P, Triagem T
WHERE (D.diagnostico > 0 AND D.id_paciente = P.id) AND (T.id_paciente = P.id AND T.colesterol > (SELECT AVG(Triagem.colesterol) FROM Triagem));

74

In [30]:
SELECT COUNT(*)
FROM Diagnostico D, Paciente P, Triagem T
WHERE (D.diagnostico > 0 AND D.id_paciente = P.id) AND (T.id_paciente = P.id AND T.colesterol > 190.0);

123

In [32]:
SELECT AVG(Triagem.glicose) FROM Triagem

0.1485148514851485

In [41]:
SELECT COUNT(*)
FROM Diagnostico D, Paciente P, Triagem T
WHERE (D.diagnostico = 0 AND D.id_paciente = P.id) AND (T.id_paciente = P.id AND T.glicose = 1);

23

In [43]:
SELECT AVG(batimento_max) FROM ECG

149.6072607260726

In [45]:
SELECT COUNT(*)
FROM Diagnostico D, Paciente P, ECG E
WHERE (D.diagnostico > 0 AND D.id_paciente = P.id) AND (E.id_paciente = P.id AND E.batimento_max > (SELECT AVG(batimento_max) FROM ECG));

50

In [52]:
SELECT COUNT(*)
FROM Diagnostico D, Paciente P, ECG E
WHERE (D.diagnostico = 0 AND D.id_paciente = P.id) AND (E.id_paciente = P.id AND E.batimento_max > 140);

138

In [57]:
SELECT COUNT(*)
FROM Diagnostico D, Paciente P, ECG E
WHERE (D.diagnostico > 0 AND D.id_paciente = P.id) AND (E.id_paciente = P.id AND E.resultado = 2);

80

In [58]:
SELECT AVG(depressao_st) FROM ECG

1.0396039603960396

In [60]:
SELECT COUNT(*)
FROM Diagnostico D, Paciente P, ECG E
WHERE (D.diagnostico = 0 AND D.id_paciente = P.id) AND (E.id_paciente = P.id AND E.depressao_st > (SELECT AVG(depressao_st) FROM ECG));

41

In [61]:
SELECT COUNT(*)
FROM Diagnostico D, Paciente P, ECG E
WHERE (D.diagnostico > 0 AND D.id_paciente = P.id) AND (E.id_paciente = P.id AND E.depressao_st > (SELECT AVG(depressao_st) FROM ECG));

82